In [78]:
using StatsBase, Plots
include("../code/slim_graphs.jl")
include("../code/matrix_generator.jl")

valid_degrees (generic function with 2 methods)

## 1. Monitor the performance of leaf removal as a strategy to build basis

Build an instance with $f_2=1, f_1\approx 0$

In [127]:
f1 = 0.0
N = 8*10^4
N1 = Int(N*f1)
Q = mod1(sum(digits(2N-N1)),3)
N1 = N1 + Q
N2 = N - N1
nedges = (N1+2N2)
M = Int(nedges/3)
Lambda = [N1, N2] / N
@assert sum(Lambda)==1
Rho = [0,0,1]

3-element Vector{Int64}:
 0
 0
 1

In [128]:
Ht = ldpc_matrix(N, M, nedges, Lambda, Rho, accept_multi_edges=false)
H = sparse(Ht');
using BenchmarkTools

Break down `findbasis(H, Ht) in Main at /home/crotti/sourcecoding/code/slim_graphs.jl:97` and benchmark each line

In [129]:
rowperm, colperm = leaf_removal(H, Ht)
@btime rowperm, colperm = leaf_removal(H, Ht);

  16.222 ms (106756 allocations: 16.57 MiB)


In [130]:
Hnew = H[rowperm, colperm]
@btime Hnew = H[rowperm, colperm];

  12.715 ms (20 allocations: 4.07 MiB)


In [131]:
@btime ut2diagGF2!($(copy(Hnew)));

  1.017 ms (0 allocations: 0 bytes)


In [132]:
ut2diagGF2!(Hnew);

In [133]:
B = [Hnew[:, size(Hnew, 1)+1:end]; I]
@btime B = [Hnew[:, size(Hnew, 1)+1:end]; I];

  3.103 ms (31 allocations: 14.17 MiB)


In [134]:
indep = colperm[size(H,1)+1:end]
@btime indep = colperm[size(H,1)+1:end];

  24.571 μs (6 allocations: 208.53 KiB)


In [135]:
@btime B[invperm(colperm),:], indep;

  37.252 ms (23 allocations: 10.72 MiB)


### Scaling of the computation time

In [ ]:
f1 = 0.0
Rho = [0,0,1]
Ns = round.(Int,10 .^ (3:0.5:6))
navg = 10
times = zeros(length(Ns))
memory = zeros(length(Ns))

for (i,N) in pairs(Ns)
    println("## Size $i of ", length(Ns))
    N1 = Int(N*f1)
    Q = mod1(sum(digits(2N-N1)),3)
    N1 = N1 + Q
    N2 = N - N1
    nedges = (N1+2N2)
    M = Int(nedges/3)
    Lambda = [N1, N2] / N
    @assert sum(Lambda)==1
    Ht = ldpc_matrix(N, M, nedges, Lambda, Rho, accept_multi_edges=false)
    H = sparse(Ht')
    for j in 1:navg
        t = @timed (B, indep = findbasis(H, Ht))
        times[i] += t[2]/navg
        memory[i] += t[3]/navg
    end
end

## Size 1 of 7
## Size 2 of 7
## Size 3 of 7
## Size 4 of 7
## Size 5 of 7
## Size 6 of 

In [ ]:
plot(Ns, times, marker=:circle, color=:blue, label="Time", legend=:topleft, foreground_color_guide=:blue)
plot!(twinx(), Ns, memory, marker=:circle, color=:red, label="Memory", yforeground_color_guide=:red)
plot!(xlabel="N", right_margin=15Plots.mm)

## 2. Observe horizontal and vertical weights

In [ ]:
f1s = 0:0.1:0.9
f1s = [0]
N = 10^6
horiz = []
vert = []
@showprogress for (i,f1) in enumerate(f1s)
    N1 = Int(N*f1)
    Q = mod1(sum(digits(2N-N1)),3)
    N1 = N1 + Q
    N2 = N - N1
    nedges = (N1+2N2)
    M = Int(nedges/3)
    Lambda = [N1, N2] / N
    @assert sum(Lambda)==1
    Rho = [0,0,1]
    Ht = ldpc_matrix(N, M, nedges, Lambda, Rho, accept_multi_edges=false)
    H = sparse(Ht')
    B = findbasis_partial(H, Ht)
    push!(horiz, vec(sum(B,dims=2)))
    push!(vert, vec(sum(B,dims=1)))
end

In [ ]:
colors = cgrad(:cool, length(f1s)+1, categorical = true)
plv = plot()
for i in eachindex(f1s)
    hv = fit(Histogram, vert[i],nbins=20)
    hv = normalize(hv)
    plot!(plv, hv.edges[1][1:end-1], hv.weights, label="f1=$(f1s[i])", color=colors[i])
#     histogram!(plv, vert[i], yaxis=(:log10,(1e-5,Inf)), normalize=:pdf)
end
plot(plv, yaxis=(:log10,(1e-8,Inf)), xaxis=(:log10,(1,Inf)), title="Vertical weight",
    legend=:outertopright)

In [ ]:
colors = cgrad(:cool, length(f1s)+1, categorical = true)
plh = plot()
for i in 1:length(f1s)
    hh = fit(Histogram,horiz[i],nbins=10^4)
    hh = normalize(hh)
    plot!(plh, hh.edges[1][1:end-1], hh.weights, label="f1=$(f1s[i])", color=colors[i])
#     histogram!(plv, vert[i], yaxis=(:log10,(1e-5,Inf)), normalize=:pdf)
end
plot(plh, yaxis=(:log10,(1e-8,Inf)), xaxis=(:log10,(1,Inf)), title="Horizontal weight",
    legend=:outertopright)